<a href="https://colab.research.google.com/github/pavaris-pm/ManiCLIP/blob/for-colab/maniclip_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `ManiCLIP`: Multi-Attribute Face Editing
- dedicated training and architecture setup by Mr. Pavaris Ruangchutiphophan (`github`: pavaris-pm)

-------------------------------------------


## FAQs
- always read it before asking up any question, if it is already in here, i will not answer it even you contact me about it

### Q1.) Did this notebook is an implementation of ManiCLIP ?

`Ans`: Yes, however, i'm mainly coding in GitHub codespace so that whatever changes i've made will be pushed directly to the repository and will use Google colab to execute the script instead. It is waste of my time to implement it on both environment. Therefore, if you have any question, just lookup the code in the repo, i already leave a comment on it already.

---------------------------------------


### Q2.) I couldn't run it on my environment, how could i do it ?

`Ans`: if you're working on codespace e.g. github codespace, vscode etc.. I recommend you to create a new environment via `env` command by create it from `environment.yaml` file that on this repo. Since the version of `torchvision` library is somewhat needed a specific version (many GANs implementation mainly rely on `vutils` function), for colab, please make sure that you've clone the correct repo (branch name: `for-colab`)



```cli
!git clone --branch for-colab https://github.com/pavaris-pm/ManiCLIP.git
```



-----------------------------------------


  ### Q3.) What is an expected result of this implementation ?

  `Ans`: this is one of the new way of face editing task since many models always handle with only single-attribute only (except with conditional gan that seems much on give a condition, however, its training scheme always look heavy if we training it on the attributes/latent as always) with that ManiCLIP coming out to handle multi-face attribute with a very new training scheme (like think outside-the-box manner)


  Apart from that, i projected to improve it in order to handle with low-resource language so that it will be accessed by most of people to build their own generation models from now on since i saw a possible implementation to get this done, and now i', currently working on it (it'll be done if i'm not that lazy 😭).


  -----------------------------------------
  
   ### Q4.) since you pull some data from Google drive, will the resource on the google drive published?
   ```python
  from google.colab import drive
  drive.mount('/content/drive')
   ```

  `Ans`: No. I will not share all of my resources used to train the model. Everything buildup in this repo can be followed based on the `readme.md` file. Just follow it can replicate the paper as well.

  -----------------------------------------

  ### Q5.) What if i have a question about this model ?

  `Ans`: i'm a very active developer on github, any questions coming up from this implementation. It is better to open up an issue on my repo and do not forget to `@pavaris-pm` for notify me to read it, i will answer it immediately.

  -----------------------------------------

# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone --branch for-colab https://github.com/pavaris-pm/ManiCLIP.git
# to obtain dataset, it will decompressed into ./
#!unzip -q /content/drive/MyDrive/SpatialAttGAN/CelebA/Img/img_align_celeba.zip

Cloning into 'ManiCLIP'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 53 (delta 9), reused 0 (delta 0), pack-reused 35
Receiving objects: 100% (53/53), 57.95 MiB | 14.18 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [1]:
!pip -q install open_clip_torch
!pip -q install clip
!pip -q install ninja
!pip -q install git+https://github.com/openai/CLIP.git
!pip -q install multilingual-clip torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
# directory setup
import os
folder_path = "/content/pretrained/"  # Replace with the actual path you want
data_path = "/content/data/"
# Use os.makedirs with exist_ok=True to create the folder only if it doesn't exist
os.makedirs(folder_path, exist_ok=True)
os.makedirs(data_path, exist_ok=True)

In [ ]:
# to format the directory structure
!cp /content/drive/MyDrive/SpatialAttGAN/OneDrive_1_09-12-2023/* /content/pretrained/
!cp /content/ManiCLIP/data/* /content/data/
!cp /content/drive/MyDrive/SpatialAttGAN/CelebA/Anno/list_attr_celeba.txt /content/data/

In [ ]:
clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

# Model Training
- Note that ManiCLIP offered a new traning scheme that is very much impressive, better to take a look at the paper
- requirements for run the script must be created

In [ ]:
# script argument lookup (can be customed on your own)
!python /content/ManiCLIP/train.py --help

In [ ]:
%%time
!python /content/ManiCLIP/train.py --epochs 1 --loss_id_weight 0.05 --loss_w_norm_weight 0.1 --loss_clip_weight 1.0 --loss_face_norm_weight 0.05 --loss_minmaxentropy_weight 0.2 --loss_face_bg_weight 1 --task_name name --decouple --part_sample_num 3

2023-12-09 13:18:48.696794: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 13:18:48.696852: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 13:18:48.696897: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 13:18:49.848968: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/ManiCLIP/train.py:127: UserWarning: You have chosen to seed training. This will turn on the CUDNN deterministic setting, which can slow down your training considerably! You may see unexpected behavior when restarting from checkpo

# Generation

In [ ]:
!python /content/ManiCLIP/generate.py \
--model_path pretrained/pretrained_edit_model.pth.tar \
--text "this person has grey hair. he has red lips, and black mustache." \
--gen_num 3

2023-12-09 14:12:13.948777: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-09 14:12:13.948840: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-09 14:12:13.948879: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-09 14:12:15.627390: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/ManiCLIP/external/stylegan2/op/conv2d_gradfix.py:88: UserWarning: conv2d_gradfix not supported on PyTorch 2.1.0+cu118. Falling back to torch.nn.functional.conv2d().
  warnings.warn(


# `Future Work`: Low-Resource ManiCLIP 🚧

In [ ]:
# experiment of ManiCLIP on Thai Language
!python /content/ManiCLIP/generate.py \
--model_path pretrained/pretrained_edit_model.pth.tar \
--text 'คนนี้ผมหงอก ปากแดง มีหนวดดำ' \
--gen_num 1

In [ ]:
# experiment of ManiCLIP on Japanese Language
!python /content/ManiCLIP/generate.py \
--model_path pretrained/pretrained_edit_model.pth.tar \
--text 'この人は白髪、赤い唇、黒い口ひげを持っています。' \
--gen_num 1

In [ ]:
# experiment of ManiCLIP on Korean Language
!python /content/ManiCLIP/generate.py \
--model_path pretrained/pretrained_edit_model.pth.tar \
--text '이 사람은 회색 머리에 붉은 입술, 검은 콧수염을 갖고 있습니다.' \
--gen_num 1